In [1]:
from collections import namedtuple
import random
from copy import deepcopy
from typing import Callable
from itertools import accumulate
from operator import xor

## The *Nim* and *Nimply* classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects   
   
    
   
# x= Nim(5)
# x.nimming(2, 3) #leave 3 objects from the row 2 (the first row is row 0)
# print(x._rows)

In [4]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

### Some strategies used for the evaluation of the evolved strategy

In [5]:
def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]


In [6]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [7]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [8]:
def reverse_gabriele(state: Nim) -> Nimply:
    """Pick always the minumum possible number (1) of the maximum row"""
    data= cook_status(state)
    return Nimply(data["longest_row"], 1)

In [9]:
def giovanni(state: Nim) -> Nimply:
    data= cook_status(state)
    """If active rows are even, take 1 from the longest; else take all from the shortest"""
    if(data["active_rows_number"] % 2 ==0):
        return Nimply(data["longest_row"], 1)
    else:
        return Nimply(data["shortest_row"], state.rows[data["shortest_row"]])

In [10]:
def my_very_silly(state: Nim) -> Nimply:
    """Do actions to try to lose the majority of the times"""
    data= cook_status(state)
    if(data["active_rows_number"]>=3):
        return Nimply(data["shortest_row"], 1)
    elif(data["active_rows_number"]==2):
        return Nimply(data["longest_row"], state.rows[data["longest_row"]])
    elif(data["active_rows_number"]==1):
        if(state.rows[data["shortest_row"]]>1):
            return Nimply(data["shortest_row"], state.rows[data["shortest_row"]]-1)
        else:
            return Nimply(data["shortest_row"], 1)

In [11]:
def my_smart(state: Nim) -> Nimply:
    """Do actions to try to win the majority of the times"""
    data= cook_status(state)
    if(data["active_rows_number"]>=3):
        return Nimply(data["shortest_row"], 1)
    elif(data["active_rows_number"]==2 and state.rows[data["longest_row"]]>1):
        return Nimply(data["longest_row"], state.rows[data["longest_row"]]-1)
    elif(data["active_rows_number"]==2 and state.rows[data["longest_row"]]<=1):
        return Nimply(data["longest_row"], 1)
    elif(data["active_rows_number"]==1):
        return Nimply(data["shortest_row"], state.rows[data["shortest_row"]])


### My evolved hardcoded 

In [12]:
# function in which the ply are chosen on the basis of the values of the genes of the genome
def my_evolved_hard_coded(state: Nim, genome) -> Nimply:
    data= cook_status(state)
    if(data["active_rows_number"]>=genome[0]):
        if(state.rows[data["longest_row"]]>1):
            return Nimply(data["longest_row"], int(state.rows[data["longest_row"]]/2))
        else:
            return Nimply(data["longest_row"], 1)
    elif(data["active_rows_number"]==genome[1] and state.rows[data["longest_row"]]>genome[3]):
        if(state.rows[data["longest_row"]]>1):
            return Nimply(data["longest_row"], state.rows[data["longest_row"]]-1)
        else:
            return Nimply(data["longest_row"], 1)
    elif(data["active_rows_number"]==genome[1] and state.rows[data["longest_row"]]<=genome[3]):
        return Nimply(data["longest_row"], 1)
    elif(data["active_rows_number"]==genome[2]):
        return Nimply(data["shortest_row"], state.rows[data["shortest_row"]])
    elif(data["active_rows_number"]==genome[4]):
        if(state.rows[data["shortest_row"]]>1):
            return Nimply(data["shortest_row"], state.rows[data["shortest_row"]]-1)
        else:
            return Nimply(data["shortest_row"], 1)
    else:
        return Nimply(data["longest_row"], 1)


### Parameters for the evaluation and the generation

In [13]:
NUM_GENERATIONS = 200
NUM_MATCHES = 10
NIM_SIZE = 8
NUM_GENES = 5

### Evaluation (to compute the fitness)

In [14]:
# The evaluation is used to evaluate a genome, so to compute its fitness: a genome is a tuple of 5 elements (genes) and these are used in 
# the function my_evolved_hard_coded as parameters of equalities and inequalities, for instance: "if number of active rows is greater than
#  genome[0], do this", etc. 
# In the evaluation, there are 10 matches, 5 in which the my_evolved_hard_coded starts first and 5 in which the my_evolved_hard_coded starts
# second against 5 hard coded rules of different difficulties (so 2 match for each hard coded) (Doing more than 2 match for each hard coded is
# useless because doing one match in which one start first and another in which the same start seconds, covers all the possibilities, because
# the hard coded are not randomic)
def evaluate(strategy: Callable, genome_ev) -> int:
    p1_wins_ev= 0
    opponent= ()
    for m in range(NUM_MATCHES):
        if (m< NUM_MATCHES/5):
            opponent = (strategy, my_very_silly) 
        elif(m>= NUM_MATCHES/5 and m< 2*NUM_MATCHES/5):
            opponent = (strategy, reverse_gabriele) 
        elif(m>= 2*NUM_MATCHES/5 and m< 3*NUM_MATCHES/5):
            opponent = (strategy, gabriele) 
        elif(m>= 3*NUM_MATCHES/5 and m< 4*NUM_MATCHES/5):
            opponent = (strategy, giovanni)
        else:
            opponent = (strategy, my_smart)
        nim_ev = Nim(NIM_SIZE)
        if m%2== 0:           
            p1_turn_ev= True
        else:
            p1_turn_ev= False
        while nim_ev:
            if(p1_turn_ev):
                ply_ev = opponent[0](nim_ev, genome_ev)
                nim_ev.nimming(ply_ev)
                p1_turn_ev= False
            else:
                ply_ev = opponent[1](nim_ev)
                nim_ev.nimming(ply_ev)
                p1_turn_ev= True 
        if(not p1_turn_ev):
            p1_wins_ev+=1
    return p1_wins_ev  #this is the fitness: the number of matches won

### Mutation 

In [15]:
#change in a random way all the genes that compose the genome (create a new genome)
def mutation():
    nim_mu= Nim(NIM_SIZE)
    init_mu= cook_status(nim_mu)
    return tuple([random.randrange(1, NIM_SIZE), random.randrange(1, NIM_SIZE), random.randrange(1, NIM_SIZE), random.randrange(1, nim_mu._rows[init_mu["longest_row"]]), random.randrange(1, NIM_SIZE)])

### Generation

In [16]:
chosen= tuple()
max_fit= 0
for _ in range(NUM_GENERATIONS):
    g = mutation() #a mutation is a random creation of a new genome
    f= evaluate(my_evolved_hard_coded, g)
    if(f > max_fit):
        max_fit= f
        chosen= g
print("genome: ",chosen, "fitness: ", max_fit)

genome:  (7, 7, 1, 9, 5) fitness:  8


### To verify the results (debug)

In [17]:
# section to verify that the fitness is correct and to play against pure_random and optimal_strategy (to lose :))
p1_wins=0
first_move_p1= 0
chosen_genome= list(chosen)

for m in range(2): #In the first it starts first, in the second, it starts second
    x= Nim(NIM_SIZE)
    if m%2== 0:
        first_move_p1+=1
        p1_turn= True
    else:
        p1_turn= False
    print("Starting point", x._rows)
    while(x):
        if(p1_turn):
            ply= my_evolved_hard_coded(x, chosen_genome)
            x.nimming(ply)
            print("After p1 move", x._rows)
            p1_turn= False
        else:
            #ply= pure_random(x)
            #ply= giovanni(x)
            #ply= reverse_gabriele(x)
            #ply= optimal_strategy(x)
            #ply= my_smart(x)
            #ply= my_very_silly(x)
            ply= gabriele(x)
            x.nimming(ply)
            print("After p2 move", x._rows)
            p1_turn= True
    if(not p1_turn):
        p1_wins+=1
    
print("After", 2, "matches:\nP1 (my evolved hard coded) won ",p1_wins," matches (starting first ", first_move_p1," times!).\nP2 (with my hard coded) won ",2-p1_wins," matches (starting first ", 2-first_move_p1," times!).")
    
        

Starting point [1, 3, 5, 7, 9, 11, 13, 15]
After p1 move [1, 3, 5, 7, 9, 11, 13, 8]
After p2 move [0, 3, 5, 7, 9, 11, 13, 8]
After p1 move [0, 3, 5, 7, 9, 11, 7, 8]
After p2 move [0, 0, 5, 7, 9, 11, 7, 8]
After p1 move [0, 0, 5, 7, 9, 10, 7, 8]
After p2 move [0, 0, 0, 7, 9, 10, 7, 8]
After p1 move [0, 0, 0, 1, 9, 10, 7, 8]
After p2 move [0, 0, 0, 0, 9, 10, 7, 8]
After p1 move [0, 0, 0, 0, 9, 9, 7, 8]
After p2 move [0, 0, 0, 0, 0, 9, 7, 8]
After p1 move [0, 0, 0, 0, 0, 8, 7, 8]
After p2 move [0, 0, 0, 0, 0, 0, 7, 8]
After p1 move [0, 0, 0, 0, 0, 0, 7, 7]
After p2 move [0, 0, 0, 0, 0, 0, 0, 7]
After p1 move [0, 0, 0, 0, 0, 0, 0, 0]
Starting point [1, 3, 5, 7, 9, 11, 13, 15]
After p2 move [0, 3, 5, 7, 9, 11, 13, 15]
After p1 move [0, 3, 5, 7, 9, 11, 13, 8]
After p2 move [0, 0, 5, 7, 9, 11, 13, 8]
After p1 move [0, 0, 5, 7, 9, 11, 12, 8]
After p2 move [0, 0, 0, 7, 9, 11, 12, 8]
After p1 move [0, 0, 0, 1, 9, 11, 12, 8]
After p2 move [0, 0, 0, 0, 9, 11, 12, 8]
After p1 move [0, 0, 0, 0, 9, 1